In [1]:
%cd ..

G:\My Drive\0-Research-Project\MedStar\MS_CODE\FieldNN


# PreCode

In [2]:
import os
from recfldgrn.datapoint import load_df_data_from_folder
from fieldnn.utils.layerfn import traverse, convert_relational_list_to_numpy

###################### take this as given
batch_PID_order = ['P1', 'P4', 'P5', 'P6']
######################

TaskTensor_folder = 'data/ProcData/TensorFolder/Task2YearXXX'
print(TaskTensor_folder)

l = sorted([i for i in os.listdir(TaskTensor_folder) if 'Grn' in i])
# l

recfldgrn_list =['P@age-AgeNumeDftGrn',
                 'P@basicInfo-basicInfoDftGrn',
                 
                 'EC@BasicInfo-BasicDftGrn',
                 'EC@DT_min-DTDftGrn',
                 
                 'A1C@DT-DTDftGrn',
                 'A1C@V-A1CNumeDftGrn',
                 
                 'Diag@DT-DTDftGrn',
                 'Diag@Value-DiagDftGrn',
                 
                 'PNSectSent@Sentence-Tk@TknzLLMGrn']

data/ProcData/TensorFolder/Task2YearXXX


In [3]:
batch_rfg = {}

for recfldgrn in recfldgrn_list:
    
    # (1) get tensor_folder
    tensor_folder = os.path.join(TaskTensor_folder, recfldgrn)
    print(tensor_folder)

    # (2) get df_Pat and full_recfldgrn
    df_Pat = load_df_data_from_folder(tensor_folder).set_index('PID')
    full_recfldgrn = df_Pat.columns[0]
    suffix = full_recfldgrn.split('_')[-1]
    assert recfldgrn in full_recfldgrn

    # (3) load batch: TODO: convert this to DataSet and DataLoader
    df_batch = df_Pat.loc[batch_PID_order]

    # (4) tensor batch as tensor_idx
    new_full_recfldgrn = 'B-' + full_recfldgrn
    values_list = df_batch[full_recfldgrn].to_list()
    suffix = full_recfldgrn.split('_')[-1]
    # print(suffix)
    # print(new_full_recfldgrn)
    D = convert_relational_list_to_numpy(values_list, new_full_recfldgrn, suffix)
    tensor_idx = D[new_full_recfldgrn]
    
    batch_rfg[new_full_recfldgrn] = tensor_idx
    
for k, v in batch_rfg.items(): print(k, v.shape)

data/ProcData/TensorFolder/Task2YearXXX\P@age-AgeNumeDftGrn
data/ProcData/TensorFolder/Task2YearXXX\P@basicInfo-basicInfoDftGrn
data/ProcData/TensorFolder/Task2YearXXX\EC@BasicInfo-BasicDftGrn
data/ProcData/TensorFolder/Task2YearXXX\EC@DT_min-DTDftGrn
data/ProcData/TensorFolder/Task2YearXXX\A1C@DT-DTDftGrn
data/ProcData/TensorFolder/Task2YearXXX\A1C@V-A1CNumeDftGrn
data/ProcData/TensorFolder/Task2YearXXX\Diag@DT-DTDftGrn
data/ProcData/TensorFolder/Task2YearXXX\Diag@Value-DiagDftGrn
data/ProcData/TensorFolder/Task2YearXXX\PNSectSent@Sentence-Tk@TknzLLMGrn
B-P@age-AgeNumeDftGrn_wgt (4, 19)
B-P@basicInfo-basicInfoDftGrn_idx (4, 2)
B-P-EC@BasicInfo-BasicDftGrn_idx (4, 25, 2)
B-P-EC@DT_min-DTDftGrn_idx (4, 25, 7)
B-P-EC-A1C@DT-DTDftGrn_idx (4, 25, 1, 7)
B-P-EC-A1C@V-A1CNumeDftGrn_wgt (4, 25, 1, 37)
B-P-EC-Diag@DT-DTDftGrn_idx (4, 25, 22, 7)
B-P-EC-Diag@Value-DiagDftGrn_idx (4, 25, 22, 3)
B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx (4, 25, 1, 14, 121, 11)


In [4]:
full_recfldgrn_list = [k for k in batch_rfg]
full_recfldgrn_list

['B-P@age-AgeNumeDftGrn_wgt',
 'B-P@basicInfo-basicInfoDftGrn_idx',
 'B-P-EC@BasicInfo-BasicDftGrn_idx',
 'B-P-EC@DT_min-DTDftGrn_idx',
 'B-P-EC-A1C@DT-DTDftGrn_idx',
 'B-P-EC-A1C@V-A1CNumeDftGrn_wgt',
 'B-P-EC-Diag@DT-DTDftGrn_idx',
 'B-P-EC-Diag@Value-DiagDftGrn_idx',
 'B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx']

In [5]:
import torch 

RECFLD_TO_TENSOR = {}

for full_recfldgrn in full_recfldgrn_list:
    # (1) get the info_raw from batch_rfg
    info_raw = batch_rfg[full_recfldgrn]
   
    # (2) get the holder (input_idx) and holder_wgt (for nume embedding only)
    if '_idx' in full_recfldgrn:
        holder_wgt = 'Empty'
        holder = torch.LongTensor(info_raw)
    elif '_wgt' in full_recfldgrn:
        holder_wgt = torch.FloatTensor(info_raw)
        # ATTENTION: here holder_wgt could contain zeros in some valid positions.
        holder = torch.ones_like(holder_wgt).cumsum(-1).masked_fill(holder_wgt == 0, 0).long()
    else:
        raise ValueError(f'Invalid suffix "{suffix}"')

    info_dict = {'holder': holder, 'holder_wgt': holder_wgt}
    
    RECFLD_TO_TENSOR[full_recfldgrn] = info_dict
    

In [6]:
from fieldnn.dataflowfn.embedflowfn import get_EmbeddingBlock_DataFlow

from fieldnn.dataflowfn.baseflowfn import mapping_SubUnitName_to_SubUnitNNList
from fieldnn.dataflowfn.baseflowfn import get_SubUnit_Default_NNPara_List
from fieldnn.dataflowfn.baseflowfn import get_SubUnit_BasicNN_Config_List

############################################# Hyperparameters
default_BasicNNtype_To_NNName = {
    'expander': None, # will be updated according to the Grn Type
    'reducer': 'Max',
    'merger': 'Merger',
    'learner': None, # TODO: ignore this currently
    
}
#############################################

############################
embed_size = 128
process = {'activator': 'gelu',
           'dropout': {'p': 0.5, 'inplace': False},
           'layernorm': {'eps': 1e-05, 'elementwise_affine': True}}
############################


default_SubUnitName = 'E'
fldgrn_folder = 'data/ProcData/FldGrnInfo'
learner_default_dict = {} # To update it in the future. 




df_SubUnit = get_EmbeddingBlock_DataFlow(full_recfldgrn_list, default_SubUnitName)

s = df_SubUnit.apply(lambda x: mapping_SubUnitName_to_SubUnitNNList(x['SubUnitName'], 
                                                                    x['input_names'],
                                                                    default_BasicNNtype_To_NNName), 
                    axis = 1)
df_SubUnit['SubUnit_BasicNN_List'] = s
s = df_SubUnit.apply(lambda x: get_SubUnit_Default_NNPara_List(x['SubUnit_BasicNN_List'], 
                                                               x['input_names'],
                                                               fldgrn_folder, 
                                                               learner_default_dict), axis = 1)

df_SubUnit['SubUnit_DefaultBasicNN_List'] = s



s = df_SubUnit.apply(lambda x: get_SubUnit_BasicNN_Config_List(x['SubUnit_BasicNN_List'], 
                                                               x['SubUnit_DefaultBasicNN_List'], 
                                                               x['input_names'], 
                                                               x['output_name'], 
                                                                embed_size, 
                                                                process, 
                                                               ), axis = 1)

df_SubUnit['SubUnit_BasicNN_Config_List'] = s
df_SubUnit

,SubUnitName,input_names,output_name,output_layerid,SubUnit_BasicNN_List,SubUnit_DefaultBasicNN_List,SubUnit_BasicNN_Config_List
0,E,[B-P@age-AgeNumeDftGrn_wgt],B-P@age-AgeNumeDft,3,[expander-NumeEmbed],"[{'vocab_tokenizer': [1, 1, 1, 1, 1, 1, 1, 1, ...","[{'nn_type_nn_name': 'expander-NumeEmbed', 'Ba..."
1,E,[B-P@basicInfo-basicInfoDftGrn_idx],B-P@basicInfo-basicInfoDft,3,[expander-CateEmbed],"[{'vocab_tokenizer': {'_padding': 0, 'Male': 1...","[{'nn_type_nn_name': 'expander-CateEmbed', 'Ba..."
2,E,[B-P-EC@BasicInfo-BasicDftGrn_idx],B-P-EC@BasicInfo-BasicDft,4,[expander-CateEmbed],"[{'vocab_tokenizer': {'_padding': 0, 'X': 1, '...","[{'nn_type_nn_name': 'expander-CateEmbed', 'Ba..."
3,E,[B-P-EC@DT_min-DTDftGrn_idx],B-P-EC@DT_min-DTDft,4,[expander-CateEmbed],"[{'vocab_tokenizer': {'_padding': 0, '_missing...","[{'nn_type_nn_name': 'expander-CateEmbed', 'Ba..."
4,E,[B-P-EC-A1C@DT-DTDftGrn_idx],B-P-EC-A1C@DT-DTDft,5,[expander-CateEmbed],"[{'vocab_tokenizer': {'_padding': 0, '_missing...","[{'nn_type_nn_name': 'expander-CateEmbed', 'Ba..."
5,E,[B-P-EC-A1C@V-A1CNumeDftGrn_wgt],B-P-EC-A1C@V-A1CNumeDft,5,[expander-NumeEmbed],"[{'vocab_tokenizer': [1, 1, 1, 1, 1, 1, 1, 1, ...","[{'nn_type_nn_name': 'expander-NumeEmbed', 'Ba..."
6,E,[B-P-EC-Diag@DT-DTDftGrn_idx],B-P-EC-Diag@DT-DTDft,5,[expander-CateEmbed],"[{'vocab_tokenizer': {'_padding': 0, '_missing...","[{'nn_type_nn_name': 'expander-CateEmbed', 'Ba..."
7,E,[B-P-EC-Diag@Value-DiagDftGrn_idx],B-P-EC-Diag@Value-DiagDft,5,[expander-CateEmbed],"[{'vocab_tokenizer': {'_padding': 0, '_missing...","[{'nn_type_nn_name': 'expander-CateEmbed', 'Ba..."
8,E,[B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzL...,B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLM,7,[expander-LLMEmbed],[{'vocab_tokenizer': PreTrainedTokenizerFast(n...,"[{'nn_type_nn_name': 'expander-LLMEmbed', 'Bas..."


In [7]:
from fieldnn.module.embedblock import EmbedBlockLayer

EmbedBlock = EmbedBlockLayer(df_SubUnit)

RECFLD_TO_EMBEDTESNOR = EmbedBlock(RECFLD_TO_TENSOR)

for k, v in RECFLD_TO_EMBEDTESNOR.items():
    print(k, v['info'].shape)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


B-P@age-AgeNumeDft torch.Size([4, 19, 128])
B-P@basicInfo-basicInfoDft torch.Size([4, 2, 128])
B-P-EC@BasicInfo-BasicDft torch.Size([4, 25, 2, 128])
B-P-EC@DT_min-DTDft torch.Size([4, 25, 7, 128])
B-P-EC-A1C@DT-DTDft torch.Size([4, 25, 1, 7, 128])
B-P-EC-A1C@V-A1CNumeDft torch.Size([4, 25, 1, 37, 128])
B-P-EC-Diag@DT-DTDft torch.Size([4, 25, 22, 7, 128])
B-P-EC-Diag@Value-DiagDft torch.Size([4, 25, 22, 3, 128])
B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLM torch.Size([4, 25, 1, 14, 121, 11, 128])


# Get Input Tensor

In [8]:
for full_recfld, info_dict in RECFLD_TO_EMBEDTESNOR.items():
    print(full_recfld, info_dict['info'].shape)

B-P@age-AgeNumeDft torch.Size([4, 19, 128])
B-P@basicInfo-basicInfoDft torch.Size([4, 2, 128])
B-P-EC@BasicInfo-BasicDft torch.Size([4, 25, 2, 128])
B-P-EC@DT_min-DTDft torch.Size([4, 25, 7, 128])
B-P-EC-A1C@DT-DTDft torch.Size([4, 25, 1, 7, 128])
B-P-EC-A1C@V-A1CNumeDft torch.Size([4, 25, 1, 37, 128])
B-P-EC-Diag@DT-DTDft torch.Size([4, 25, 22, 7, 128])
B-P-EC-Diag@Value-DiagDft torch.Size([4, 25, 22, 3, 128])
B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLM torch.Size([4, 25, 1, 14, 121, 11, 128])


# Get df_SubUnit 

In [9]:
from fieldnn.dataflowfn.reprflowfn import get_Repr_dataflow_table
from fieldnn.dataflowfn.reprflowfn import update_df_Repr_dataflow
from fieldnn.dataflowfn.reprflowfn import update_df_Repr_dataflow_completename
from fieldnn.dataflowfn.reprflowfn import get_Repr_SubUnit_List

In [10]:
full_recfldgrn_list = [i for i in RECFLD_TO_EMBEDTESNOR]
full_recfldgrn_list

['B-P@age-AgeNumeDft',
 'B-P@basicInfo-basicInfoDft',
 'B-P-EC@BasicInfo-BasicDft',
 'B-P-EC@DT_min-DTDft',
 'B-P-EC-A1C@DT-DTDft',
 'B-P-EC-A1C@V-A1CNumeDft',
 'B-P-EC-Diag@DT-DTDft',
 'B-P-EC-Diag@Value-DiagDft',
 'B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLM']

In [11]:
df_dataflow = get_Repr_dataflow_table(full_recfldgrn_list)
# df_dataflow

df_dataflow_new = update_df_Repr_dataflow(df_dataflow, style = 'ReducerOnly')
# df_dataflow_new

df_dataflow_new = update_df_Repr_dataflow(df_dataflow, style = 'Reducer&Merger')
df_dataflow_new

,7,6,5,4,3,2
recfldgrn,,,,,,
P@age-AgeNumeDft,NaN,NaN,NaN,NaN,B-P@age-AgeNumeDft,B-P@age
P@basicInfo-basicInfoDft,NaN,NaN,NaN,NaN,B-P@basicInfo-basicInfoDft,B-P@basicInfo
EC@BasicInfo-BasicDft,NaN,NaN,NaN,B-P-EC@BasicInfo-BasicDft,B-P-EC@BasicInfo,NaN
EC@DT_min-DTDft,NaN,NaN,NaN,B-P-EC@DT_min-DTDft,B-P-EC@DT_min,NaN
A1C@DT-DTDft,NaN,NaN,B-P-EC-A1C@DT-DTDft,B-P-EC-A1C@DT,NaN,NaN
A1C@V-A1CNumeDft,NaN,NaN,B-P-EC-A1C@V-A1CNumeDft,B-P-EC-A1C@V,NaN,NaN
(Merge)A1C@DT&V,NaN,NaN,NaN,B-P-EC-A1C,B-P-EC@A1C,NaN
Diag@DT-DTDft,NaN,NaN,B-P-EC-Diag@DT-DTDft,B-P-EC-Diag@DT,NaN,NaN
Diag@Value-DiagDft,NaN,NaN,B-P-EC-Diag@Value-DiagDft,B-P-EC-Diag@Value,NaN,NaN


In [12]:
df_dataflow = df_dataflow_new.copy()

df_SubUnit = get_Repr_SubUnit_List(df_dataflow)
df_SubUnit

,SubUnitName,input_names,output_name,input_layerid,output_layerid
0,R,[B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk],B-P-EC-PN-PNSect-PNSectSent,7,6
1,R,[B-P-EC-PN-PNSect-PNSectSent],B-P-EC-PN-PNSect,6,5
2,R,[B-P-EC-A1C@DT-DTDft],B-P-EC-A1C@DT,5,4
3,R,[B-P-EC-A1C@V-A1CNumeDft],B-P-EC-A1C@V,5,4
4,R,[B-P-EC-Diag@DT-DTDft],B-P-EC-Diag@DT,5,4
5,R,[B-P-EC-Diag@Value-DiagDft],B-P-EC-Diag@Value,5,4
6,R,[B-P-EC-PN-PNSect],B-P-EC-PN,5,4
7,MR,"[B-P-EC-A1C@DT, B-P-EC-A1C@V]",B-P-EC-A1C,4,4
8,MR,"[B-P-EC-Diag@DT, B-P-EC-Diag@Value]",B-P-EC-Diag,4,4
9,R,[B-P-EC@BasicInfo-BasicDft],B-P-EC@BasicInfo,4,3


# get df_SubUnit Extra Info

## SubUnit Name to NN List

* E --> [E]
* MR --> [M, R]


In [13]:
from fieldnn.dataflowfn.baseflowfn import mapping_SubUnitName_to_SubUnitNNList

def mapping_SubUnitName_to_SubUnitNNList(SubUnitName, input_names, 
                                         default_BasicNNtype_To_NNName):
    SubUnitNNList = []
    for idx, NAME in enumerate(SubUnitName):
        if NAME == 'E':
            nn_type = 'expander'
            input_name = input_names[0]
            
            assert 'Grn' in input_name
            assert len(input_names) == 1
            
            if 'Tknz' in input_name:
                nn_name = 'LLMEmbed'
            elif '_wgt' in input_name:
                nn_name = 'NumeEmbed'
            else:
                nn_name = 'CateEmbed'
            
        elif NAME == 'R':
            if idx == 0: assert len(input_names) == 1
            nn_type = 'reducer'
            nn_name = default_BasicNNtype_To_NNName[nn_type]
            
        elif NAME == 'M':
            nn_type = 'merger'
            nn_name = default_BasicNNtype_To_NNName[nn_type]
            
        elif NAME == 'L':
            assert len(input_names) == 1
            nn_type = 'learner'
            nn_name = default_BasicNNtype_To_NNName[nn_type]
            
        else:
            raise ValueError(f'The BasicNN is not correct {NAME}')
        
        SubUnitNNList.append(nn_type + '-' + nn_name)
    return SubUnitNNList

In [14]:
############################################# Hyperparameters
default_BasicNNtype_To_NNName = {
    'expander': None, # will be updated according to the Grn Type
    'reducer': 'Max',
    'merger': 'Merger',
    'learner': None, # TODO: ignore this currently
    
}
#############################################

s = df_SubUnit.apply(lambda x: mapping_SubUnitName_to_SubUnitNNList(x['SubUnitName'], 
                                                                    x['input_names'],
                                                                    default_BasicNNtype_To_NNName), 
                    axis = 1)


df_SubUnit['SubUnit_BasicNN_List'] = s
df_SubUnit

,SubUnitName,input_names,output_name,input_layerid,output_layerid,SubUnit_BasicNN_List
0,R,[B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk],B-P-EC-PN-PNSect-PNSectSent,7,6,[reducer-Max]
1,R,[B-P-EC-PN-PNSect-PNSectSent],B-P-EC-PN-PNSect,6,5,[reducer-Max]
2,R,[B-P-EC-A1C@DT-DTDft],B-P-EC-A1C@DT,5,4,[reducer-Max]
3,R,[B-P-EC-A1C@V-A1CNumeDft],B-P-EC-A1C@V,5,4,[reducer-Max]
4,R,[B-P-EC-Diag@DT-DTDft],B-P-EC-Diag@DT,5,4,[reducer-Max]
5,R,[B-P-EC-Diag@Value-DiagDft],B-P-EC-Diag@Value,5,4,[reducer-Max]
6,R,[B-P-EC-PN-PNSect],B-P-EC-PN,5,4,[reducer-Max]
7,MR,"[B-P-EC-A1C@DT, B-P-EC-A1C@V]",B-P-EC-A1C,4,4,"[merger-Merger, reducer-Max]"
8,MR,"[B-P-EC-Diag@DT, B-P-EC-Diag@Value]",B-P-EC-Diag,4,4,"[merger-Merger, reducer-Max]"
9,R,[B-P-EC@BasicInfo-BasicDft],B-P-EC@BasicInfo,4,3,[reducer-Max]


## Get One BasicNN Config

In [15]:
from fieldnn.configfn.expanderfn import get_expander_para
from fieldnn.configfn.mergerfn import get_merger_para   
from fieldnn.configfn.reducerfn import get_reducer_para 


def generate_BasicNN_Config(nn_type_nn_name, 
                            input_names_nnlvl, 
                            default_nnpara, 
                            embed_size, 
                            process):
    '''
        please notince here, this function is not the final version yet.
    '''
    nn_type, nn_name = nn_type_nn_name.split('-')

    if nn_type == 'expander':
        
        assert len(input_names_nnlvl) == 1
        fld = input_names_nnlvl[0].split('-')[-1]
        # Get the output_name_nnlvl
        output_name_nnlvl = input_names_nnlvl[0].split('Grn')[0]
        
        # Get the input_size and output_size
        input_size = None
        output_size = embed_size 
        
        # Get the postprocess
        postprocess = process
        
        # Derive the para
        vocab_tokenizer = default_nnpara['vocab_tokenizer']
        init = default_nnpara['init']
        para = get_expander_para(nn_name, default_nnpara, 
                                 embed_size, vocab_tokenizer, init, postprocess)

    elif nn_type == 'reducer': 
        
        assert len(input_names_nnlvl) == 1
        fld = input_names_nnlvl[0].split('-')[-1]
        # Get the output_name_nnlvl
        output_name_nnlvl = input_names_nnlvl[0].replace('-' + fld, '@' + fld)

        # Get the para for the NN layer
        nn_para = default_nnpara # this will be updated.

        # Get the input_size
        input_size = embed_size

        # Get the output_size
        if nn_name.lower() == 'concat':
            # this types of merger only happened in the R in MLRL. 
            # you can skip this part safely if you haven't encountered M. 
            fld_childflds = input_names_nnlvl[0].split('-')[-1]
            assert '@' in fld_childflds
            childflds = fld_childflds.split('@')[-1]
            childflds = childflds.split('&')
            output_size = len(childflds) * embed_size
        else:
            # usually the most case. 
            output_size = embed_size 

        # Get the postprocess
        postprocess = process

        # Derive the para
        para = get_reducer_para(nn_name, default_nnpara, input_size, output_size, postprocess)  
        
    elif nn_type == 'merger':
        # generate the output name
        
        assert len(input_names_nnlvl) > 1
        # input_names_nnlvl: ['B-P-EC-A1C@DT', 'B-P-EC-A1C@V']
        
        childflds = [i.split('-')[-1] for i in input_names_nnlvl]
        # childflds: ['A1C@DT', 'A1C@V']
        
        fld_childflds = '&'.join([i.split('@')[-1] for i in childflds])
        # fld_childflds: DT&V
        
        output_prefix = input_names_nnlvl[0].replace('@' + childflds[0].split('@')[-1], '')
        # output_prefix: B-P-EC-A1C
        
        output_name_nnlvl = output_prefix + '-' + fld_childflds
        # output_name_nnlvl: B-P-EC-A1C-DT&V
        
        # Prepare the para for the NN layer
        nn_para = default_nnpara # this will be updated.

        # Get the input_size
        input_size = embed_size
        output_size = embed_size

        # Get the postprocess
        postprocess = process

        # Derive the para
        para = get_merger_para(nn_name, default_nnpara, input_size, output_size, postprocess) 
        
    else:
        raise ValueError(f'nn_type {nn_type} is not available yet')

    Basic_Config = {'input_names_nnlvl': input_names_nnlvl, 
                    'output_name_nnlvl': output_name_nnlvl, 
                    f'{nn_type}_para': para}
    
    return Basic_Config

## Get Default BasicNN Config List

In [16]:
from fieldnn.dataflowfn.baseflowfn import get_SubUnit_Default_NNPara_List

# get_SubUnit_Default_NNPara_List(SubUnit_BasicNN_List, SubUnit_input_names, fldgrn_folder, learner_default_dict)

In [17]:
SubUnit_info = df_SubUnit.iloc[0]
SubUnit_BasicNN_List = SubUnit_info['SubUnit_BasicNN_List']
SubUnit_input_names = SubUnit_info['input_names']
SubUnit_output_name = SubUnit_info['output_name']
fldgrn_folder = 'data/ProcData/FldGrnInfo'
learner_default_dict = {} # To update it in the future. 

print(SubUnit_BasicNN_List)

['reducer-Max']


In [18]:
SubUnit_DefaultBasicNN_List = get_SubUnit_Default_NNPara_List(SubUnit_BasicNN_List, SubUnit_input_names, 
                                                              fldgrn_folder, learner_default_dict)

SubUnit_DefaultBasicNN_List

[{}]

## Get BasicNN Config List

In [19]:
# the question is how to create the SubUnit_BasicNN_List for each SubUnit
# we will turn to this later. 


# from fieldnn.dataflowfn.baseflowfn import get_SubUnit_BasicNN_Config_List


def get_SubUnit_BasicNN_Config_List(SubUnit_BasicNN_List, 
                                    SubUnit_DefaultBasicNN_List, 
                                    SubUnit_input_names, 
                                    SubUnit_output_name, 
                                    embed_size, 
                                    process, 
                                   ):
    
    
    # TODO: also add the layer_idx in order to deal with the learn_layer_para.
    # This function also needs to be updated. 
    
    SubUnit_BasicNN_Config_List = []
    
    # print('\n\n************** SubUnit_BasicNN_List ****************')
    # print(SubUnit_BasicNN_List)
    # print(SubUnit_input_names)
    # print(SubUnit_output_name)
    for basic_nn_idx, nn_type_nn_name in enumerate(SubUnit_BasicNN_List):

        # Get the input_names_nnlvl
        if basic_nn_idx == 0:
            input_names_nnlvl = SubUnit_input_names # this assigments only works for the first iteration
        else:
            input_names_nnlvl = [output_name_nnlvl]
        
        ##############
        default_nnpara = SubUnit_DefaultBasicNN_List[basic_nn_idx] 
        ##############
        
        
        Basic_Config = generate_BasicNN_Config(nn_type_nn_name, 
                                               input_names_nnlvl, 
                                               default_nnpara, 
                                               embed_size, 
                                               process)
        output_name_nnlvl = Basic_Config['output_name_nnlvl']
        BasicNN_Config = {'nn_type_nn_name': nn_type_nn_name, 'Basic_Config': Basic_Config}
        SubUnit_BasicNN_Config_List.append(BasicNN_Config)
        
        # print('==========================')
        # print(basic_nn_idx, nn_type_nn_name)
        # print(input_names_nnlvl, '<-------- input_names_nnlvl')
        # print(output_name_nnlvl, '<-------- output_name_nnlvl')
        
        
        # also check the input_size of dim and output_size of dim

    final_output_name_nnlvl = output_name_nnlvl

    # if not SubUnit_output_name in final_output_name_nnlvl:
    #     print('xxx errors xxx')
    #     print(final_output_name_nnlvl, '<------- final_output_name_nnlvl')
    #     print(SubUnit_output_name, '<------- SubUnit_output_name')
    #     print('xxx errors xxx')
    assert SubUnit_output_name in final_output_name_nnlvl
    # print('\n************** End SubUnit_BasicNN_List ****************')
    return SubUnit_BasicNN_Config_List

In [20]:


############################
embed_size = 128
process = {'activator': 'gelu',
           'dropout': {'p': 0.5, 'inplace': False},
           'layernorm': {'eps': 1e-05, 'elementwise_affine': True}}
############################

s = get_SubUnit_BasicNN_Config_List(SubUnit_BasicNN_List, 
                                    SubUnit_DefaultBasicNN_List, 
                                    SubUnit_input_names, 
                                    SubUnit_output_name, 
                                    embed_size, 
                                    process, 
                                   )

SubUnit_BasicNN_Config_List = s

In [21]:
SubUnit_BasicNN_Config_List

[{'nn_type_nn_name': 'reducer-Max',
  'Basic_Config': {'input_names_nnlvl': ['B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk'],
   'output_name_nnlvl': 'B-P-EC-PN-PNSect-PNSectSent@Sentence@Tk',
   'reducer_para': {'nn_type': 'reducer',
    'nn_name': 'Max',
    'nn_para': {},
    'input_size': 128,
    'output_size': 128,
    'postprocess': {'activator': 'gelu',
     'dropout': {'p': 0.5, 'inplace': False},
     'layernorm': {'eps': 1e-05, 'elementwise_affine': True}}}}}]

## Config All In One

In [22]:

fldgrn_folder = 'data/ProcData/FldGrnInfo'
learner_default_dict = {} # To update it in the future. 


s = df_SubUnit.apply(lambda x: get_SubUnit_Default_NNPara_List(x['SubUnit_BasicNN_List'], 
                                                               x['input_names'],
                                                               fldgrn_folder, 
                                                               learner_default_dict), axis = 1)

df_SubUnit['SubUnit_DefaultBasicNN_List'] = s



In [23]:
############################
embed_size = 128
process = {'activator': 'gelu',
           'dropout': {'p': 0.5, 'inplace': False},
           'layernorm': {'eps': 1e-05, 'elementwise_affine': True}}
############################

In [24]:
s = df_SubUnit.apply(lambda x: get_SubUnit_BasicNN_Config_List(x['SubUnit_BasicNN_List'], 
                                                               x['SubUnit_DefaultBasicNN_List'], 
                                                               x['input_names'], 
                                                               x['output_name'], 
                                                                embed_size, 
                                                                process, 
                                                               ), axis = 1)

s

df_SubUnit['SubUnit_BasicNN_Config_List'] = s
df_SubUnit


,SubUnitName,input_names,output_name,input_layerid,output_layerid,SubUnit_BasicNN_List,SubUnit_DefaultBasicNN_List,SubUnit_BasicNN_Config_List
0,R,[B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk],B-P-EC-PN-PNSect-PNSectSent,7,6,[reducer-Max],[{}],"[{'nn_type_nn_name': 'reducer-Max', 'Basic_Con..."
1,R,[B-P-EC-PN-PNSect-PNSectSent],B-P-EC-PN-PNSect,6,5,[reducer-Max],[{}],"[{'nn_type_nn_name': 'reducer-Max', 'Basic_Con..."
2,R,[B-P-EC-A1C@DT-DTDft],B-P-EC-A1C@DT,5,4,[reducer-Max],[{}],"[{'nn_type_nn_name': 'reducer-Max', 'Basic_Con..."
3,R,[B-P-EC-A1C@V-A1CNumeDft],B-P-EC-A1C@V,5,4,[reducer-Max],[{}],"[{'nn_type_nn_name': 'reducer-Max', 'Basic_Con..."
4,R,[B-P-EC-Diag@DT-DTDft],B-P-EC-Diag@DT,5,4,[reducer-Max],[{}],"[{'nn_type_nn_name': 'reducer-Max', 'Basic_Con..."
5,R,[B-P-EC-Diag@Value-DiagDft],B-P-EC-Diag@Value,5,4,[reducer-Max],[{}],"[{'nn_type_nn_name': 'reducer-Max', 'Basic_Con..."
6,R,[B-P-EC-PN-PNSect],B-P-EC-PN,5,4,[reducer-Max],[{}],"[{'nn_type_nn_name': 'reducer-Max', 'Basic_Con..."
7,MR,"[B-P-EC-A1C@DT, B-P-EC-A1C@V]",B-P-EC-A1C,4,4,"[merger-Merger, reducer-Max]","[{}, {}]","[{'nn_type_nn_name': 'merger-Merger', 'Basic_C..."
8,MR,"[B-P-EC-Diag@DT, B-P-EC-Diag@Value]",B-P-EC-Diag,4,4,"[merger-Merger, reducer-Max]","[{}, {}]","[{'nn_type_nn_name': 'merger-Merger', 'Basic_C..."
9,R,[B-P-EC@BasicInfo-BasicDft],B-P-EC@BasicInfo,4,3,[reducer-Max],[{}],"[{'nn_type_nn_name': 'reducer-Max', 'Basic_Con..."


# SubUnit

## Module

In [25]:
from fieldnn.module.subunit import SubUnit_Layer

## Usage

In [26]:
OutputTensor_2_Repr = {}

for idx, SubUnit_info in df_SubUnit.iterrows():
    # print(SubUnit_info)
    
    output_name = SubUnit_info['output_name']
    SubUnitLayer = SubUnit_Layer(SubUnit_info)
    OutputTensor_2_Repr[output_name] = SubUnitLayer
    

In [27]:
[i for i in RECFLD_TO_EMBEDTESNOR]

['B-P@age-AgeNumeDft',
 'B-P@basicInfo-basicInfoDft',
 'B-P-EC@BasicInfo-BasicDft',
 'B-P-EC@DT_min-DTDft',
 'B-P-EC-A1C@DT-DTDft',
 'B-P-EC-A1C@V-A1CNumeDft',
 'B-P-EC-Diag@DT-DTDft',
 'B-P-EC-Diag@Value-DiagDft',
 'B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLM']

In [28]:
fld_updates_dict = {}
for i in RECFLD_TO_EMBEDTESNOR:
    layernum = len(i.split('-'))
    fld = i.split('-')[-1]
    if '@' in fld:
        # print(fld)
        neat_i = '-'.join(i.split('-')[:-1]) + '-' + fld.split('@')[0]
        # print(neat_i)
        same_neat_list = [t for t in RECFLD_TO_EMBEDTESNOR if neat_i + '@' in t]
        # print(same_neat_list)
        if len(same_neat_list) == 1: # itself
            # RECFLD_TO_EMBEDTESNOR[]
            fld_updates_dict[i] = neat_i
            
for old, new in fld_updates_dict.items():
    RECFLD_TO_EMBEDTESNOR[new] = RECFLD_TO_EMBEDTESNOR.pop(old)

In [29]:
[i for i in RECFLD_TO_EMBEDTESNOR]

['B-P@age-AgeNumeDft',
 'B-P@basicInfo-basicInfoDft',
 'B-P-EC@BasicInfo-BasicDft',
 'B-P-EC@DT_min-DTDft',
 'B-P-EC-A1C@DT-DTDft',
 'B-P-EC-A1C@V-A1CNumeDft',
 'B-P-EC-Diag@DT-DTDft',
 'B-P-EC-Diag@Value-DiagDft',
 'B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk']

In [30]:
OUTPUT_TO_TENSOR = RECFLD_TO_EMBEDTESNOR.copy()

for output_name, SubUnitLayer in OutputTensor_2_Repr.items():
    input_names = SubUnitLayer.SubUnit_input_names
    SubUnit_output_name, info_dict = SubUnitLayer(input_names, OUTPUT_TO_TENSOR)
    
    print(SubUnit_output_name)
    OUTPUT_TO_TENSOR[SubUnit_output_name] = info_dict

B-P-EC-PN-PNSect-PNSectSent
B-P-EC-PN-PNSect
B-P-EC-A1C@DT
B-P-EC-A1C@V
B-P-EC-Diag@DT
B-P-EC-Diag@Value
B-P-EC-PN
B-P-EC-A1C
B-P-EC-Diag
B-P-EC@BasicInfo
B-P-EC@DT_min
B-P-EC@A1C
B-P-EC@Diag
B-P-EC@PN
B-P-EC
B-P@age
B-P@basicInfo
B-P@EC
B-P


In [31]:
info_dict = OUTPUT_TO_TENSOR['B-P']

info_dict['info'].shape

torch.Size([4, 128])

# ReprBlock

## Module

In [32]:
import torch
# from .subunit import SubUnit_Layer


class ReprBlockLayer(torch.nn.Module):
    
    def __init__(self, df_SubUnit):
        super(ReprBlockLayer, self).__init__()
        self.df_SubUnit = df_SubUnit
        self.SubUnitDict = torch.nn.ModuleDict()
        
        for idx, SubUnit_info in df_SubUnit.iterrows():
            output_name = SubUnit_info['output_name']
            SubUnitLayer = SubUnit_Layer(SubUnit_info)
            self.SubUnitDict[output_name] = SubUnitLayer

    def forward(self, RECFLD_TO_EMBEDTESNOR):
        
        OUTPUT_TO_TENSOR = RECFLD_TO_EMBEDTESNOR.copy()
        
        for output_name, SubUnitLayer in self.SubUnitDict.items():
            input_names = SubUnitLayer.SubUnit_input_names
            SubUnit_output_name, info_dict = SubUnitLayer(input_names, OUTPUT_TO_TENSOR)
            
            assert output_name == SubUnit_output_name
            OUTPUT_TO_TENSOR[SubUnit_output_name] = info_dict
        
        return OUTPUT_TO_TENSOR
    

## Usage

In [33]:
ReprBlock = ReprBlockLayer(df_SubUnit)

# ReprBlock

In [34]:
fld_updates_dict = {}
for i in RECFLD_TO_EMBEDTESNOR:
    layernum = len(i.split('-'))
    fld = i.split('-')[-1]
    if '@' in fld:
        # print(fld)
        neat_i = '-'.join(i.split('-')[:-1]) + '-' + fld.split('@')[0]
        # print(neat_i)
        same_neat_list = [t for t in RECFLD_TO_EMBEDTESNOR if neat_i + '@' in t]
        # print(same_neat_list)
        if len(same_neat_list) == 1: # itself
            # RECFLD_TO_EMBEDTESNOR[]
            fld_updates_dict[i] = neat_i
            
for old, new in fld_updates_dict.items():
    RECFLD_TO_EMBEDTESNOR[new] = RECFLD_TO_EMBEDTESNOR.pop(old)

In [35]:
[i for i in RECFLD_TO_EMBEDTESNOR]

['B-P@age-AgeNumeDft',
 'B-P@basicInfo-basicInfoDft',
 'B-P-EC@BasicInfo-BasicDft',
 'B-P-EC@DT_min-DTDft',
 'B-P-EC-A1C@DT-DTDft',
 'B-P-EC-A1C@V-A1CNumeDft',
 'B-P-EC-Diag@DT-DTDft',
 'B-P-EC-Diag@Value-DiagDft',
 'B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk']

In [36]:
OUTPUT_TO_TENSOR = ReprBlock(RECFLD_TO_EMBEDTESNOR)
[i for i in OUTPUT_TO_TENSOR]

['B-P@age-AgeNumeDft',
 'B-P@basicInfo-basicInfoDft',
 'B-P-EC@BasicInfo-BasicDft',
 'B-P-EC@DT_min-DTDft',
 'B-P-EC-A1C@DT-DTDft',
 'B-P-EC-A1C@V-A1CNumeDft',
 'B-P-EC-Diag@DT-DTDft',
 'B-P-EC-Diag@Value-DiagDft',
 'B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk',
 'B-P-EC-PN-PNSect-PNSectSent',
 'B-P-EC-PN-PNSect',
 'B-P-EC-A1C@DT',
 'B-P-EC-A1C@V',
 'B-P-EC-Diag@DT',
 'B-P-EC-Diag@Value',
 'B-P-EC-PN',
 'B-P-EC-A1C',
 'B-P-EC-Diag',
 'B-P-EC@BasicInfo',
 'B-P-EC@DT_min',
 'B-P-EC@A1C',
 'B-P-EC@Diag',
 'B-P-EC@PN',
 'B-P-EC',
 'B-P@age',
 'B-P@basicInfo',
 'B-P@EC',
 'B-P']

# Next PreCode

## EmbedBlock

In [37]:
# %cd ..

In [38]:
import os
from recfldgrn.datapoint import load_df_data_from_folder
from fieldnn.utils.layerfn import traverse, convert_relational_list_to_numpy

###################### take this as given
batch_PID_order = ['P1', 'P4', 'P5', 'P6']
######################

TaskTensor_folder = 'data/ProcData/TensorFolder/Task2YearXXX'
print(TaskTensor_folder)

l = sorted([i for i in os.listdir(TaskTensor_folder) if 'Grn' in i])
# l

recfldgrn_list =['P@age-AgeNumeDftGrn',
                 'P@basicInfo-basicInfoDftGrn',
                 
                 'EC@BasicInfo-BasicDftGrn',
                 'EC@DT_min-DTDftGrn',
                 
                 'A1C@DT-DTDftGrn',
                 'A1C@V-A1CNumeDftGrn',
                 
                 'Diag@DT-DTDftGrn',
                 'Diag@Value-DiagDftGrn',
                 
                 'PNSectSent@Sentence-Tk@TknzLLMGrn']

data/ProcData/TensorFolder/Task2YearXXX


In [39]:
batch_rfg = {}

for recfldgrn in recfldgrn_list:
    
    # (1) get tensor_folder
    tensor_folder = os.path.join(TaskTensor_folder, recfldgrn)
    print(tensor_folder)

    # (2) get df_Pat and full_recfldgrn
    df_Pat = load_df_data_from_folder(tensor_folder).set_index('PID')
    full_recfldgrn = df_Pat.columns[0]
    suffix = full_recfldgrn.split('_')[-1]
    assert recfldgrn in full_recfldgrn

    # (3) load batch: TODO: convert this to DataSet and DataLoader
    df_batch = df_Pat.loc[batch_PID_order]

    # (4) tensor batch as tensor_idx
    new_full_recfldgrn = 'B-' + full_recfldgrn
    values_list = df_batch[full_recfldgrn].to_list()
    suffix = full_recfldgrn.split('_')[-1]
    # print(suffix)
    # print(new_full_recfldgrn)
    D = convert_relational_list_to_numpy(values_list, new_full_recfldgrn, suffix)
    tensor_idx = D[new_full_recfldgrn]
    
    batch_rfg[new_full_recfldgrn] = tensor_idx
    
for k, v in batch_rfg.items(): print(k, v.shape)

data/ProcData/TensorFolder/Task2YearXXX\P@age-AgeNumeDftGrn
data/ProcData/TensorFolder/Task2YearXXX\P@basicInfo-basicInfoDftGrn
data/ProcData/TensorFolder/Task2YearXXX\EC@BasicInfo-BasicDftGrn
data/ProcData/TensorFolder/Task2YearXXX\EC@DT_min-DTDftGrn
data/ProcData/TensorFolder/Task2YearXXX\A1C@DT-DTDftGrn
data/ProcData/TensorFolder/Task2YearXXX\A1C@V-A1CNumeDftGrn
data/ProcData/TensorFolder/Task2YearXXX\Diag@DT-DTDftGrn
data/ProcData/TensorFolder/Task2YearXXX\Diag@Value-DiagDftGrn
data/ProcData/TensorFolder/Task2YearXXX\PNSectSent@Sentence-Tk@TknzLLMGrn
B-P@age-AgeNumeDftGrn_wgt (4, 19)
B-P@basicInfo-basicInfoDftGrn_idx (4, 2)
B-P-EC@BasicInfo-BasicDftGrn_idx (4, 25, 2)
B-P-EC@DT_min-DTDftGrn_idx (4, 25, 7)
B-P-EC-A1C@DT-DTDftGrn_idx (4, 25, 1, 7)
B-P-EC-A1C@V-A1CNumeDftGrn_wgt (4, 25, 1, 37)
B-P-EC-Diag@DT-DTDftGrn_idx (4, 25, 22, 7)
B-P-EC-Diag@Value-DiagDftGrn_idx (4, 25, 22, 3)
B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx (4, 25, 1, 14, 121, 11)


In [40]:
full_recfldgrn_list = [k for k in batch_rfg]
full_recfldgrn_list

['B-P@age-AgeNumeDftGrn_wgt',
 'B-P@basicInfo-basicInfoDftGrn_idx',
 'B-P-EC@BasicInfo-BasicDftGrn_idx',
 'B-P-EC@DT_min-DTDftGrn_idx',
 'B-P-EC-A1C@DT-DTDftGrn_idx',
 'B-P-EC-A1C@V-A1CNumeDftGrn_wgt',
 'B-P-EC-Diag@DT-DTDftGrn_idx',
 'B-P-EC-Diag@Value-DiagDftGrn_idx',
 'B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx']

In [41]:
import torch 

RECFLD_TO_TENSOR = {}

for full_recfldgrn in full_recfldgrn_list:
    # (1) get the info_raw from batch_rfg
    info_raw = batch_rfg[full_recfldgrn]
   
    # (2) get the holder (input_idx) and holder_wgt (for nume embedding only)
    if '_idx' in full_recfldgrn:
        holder_wgt = 'Empty'
        holder = torch.LongTensor(info_raw)
    elif '_wgt' in full_recfldgrn:
        holder_wgt = torch.FloatTensor(info_raw)
        # ATTENTION: here holder_wgt could contain zeros in some valid positions.
        holder = torch.ones_like(holder_wgt).cumsum(-1).masked_fill(holder_wgt == 0, 0).long()
    else:
        raise ValueError(f'Invalid suffix "{suffix}"')

    info_dict = {'holder': holder, 'holder_wgt': holder_wgt}
    
    RECFLD_TO_TENSOR[full_recfldgrn] = info_dict
    

In [42]:
from fieldnn.dataflowfn.embedflowfn import get_EmbeddingBlock_DataFlow

from fieldnn.dataflowfn.baseflowfn import mapping_SubUnitName_to_SubUnitNNList
from fieldnn.dataflowfn.baseflowfn import get_SubUnit_Default_NNPara_List
from fieldnn.dataflowfn.baseflowfn import get_SubUnit_BasicNN_Config_List

############################################# Hyperparameters
default_BasicNNtype_To_NNName = {
    'expander': None, # will be updated according to the Grn Type
    'reducer': 'Max',
    'merger': 'Merger',
    'learner': None, # TODO: ignore this currently
    
}
#############################################

############################
embed_size = 128
process = {'activator': 'gelu',
           'dropout': {'p': 0.5, 'inplace': False},
           'layernorm': {'eps': 1e-05, 'elementwise_affine': True}}
############################


default_SubUnitName = 'E'
fldgrn_folder = 'data/ProcData/FldGrnInfo'
learner_default_dict = {} # To update it in the future. 




df_SubUnit = get_EmbeddingBlock_DataFlow(full_recfldgrn_list, default_SubUnitName)

s = df_SubUnit.apply(lambda x: mapping_SubUnitName_to_SubUnitNNList(x['SubUnitName'], 
                                                                    x['input_names'],
                                                                    default_BasicNNtype_To_NNName), 
                    axis = 1)
df_SubUnit['SubUnit_BasicNN_List'] = s
s = df_SubUnit.apply(lambda x: get_SubUnit_Default_NNPara_List(x['SubUnit_BasicNN_List'], 
                                                               x['input_names'],
                                                               fldgrn_folder, 
                                                               learner_default_dict), axis = 1)

df_SubUnit['SubUnit_DefaultBasicNN_List'] = s



s = df_SubUnit.apply(lambda x: get_SubUnit_BasicNN_Config_List(x['SubUnit_BasicNN_List'], 
                                                               x['SubUnit_DefaultBasicNN_List'], 
                                                               x['input_names'], 
                                                               x['output_name'], 
                                                                embed_size, 
                                                                process, 
                                                               ), axis = 1)

df_SubUnit['SubUnit_BasicNN_Config_List'] = s
df_SubUnit

,SubUnitName,input_names,output_name,output_layerid,SubUnit_BasicNN_List,SubUnit_DefaultBasicNN_List,SubUnit_BasicNN_Config_List
0,E,[B-P@age-AgeNumeDftGrn_wgt],B-P@age-AgeNumeDft,3,[expander-NumeEmbed],"[{'vocab_tokenizer': [1, 1, 1, 1, 1, 1, 1, 1, ...","[{'nn_type_nn_name': 'expander-NumeEmbed', 'Ba..."
1,E,[B-P@basicInfo-basicInfoDftGrn_idx],B-P@basicInfo-basicInfoDft,3,[expander-CateEmbed],"[{'vocab_tokenizer': {'_padding': 0, 'Male': 1...","[{'nn_type_nn_name': 'expander-CateEmbed', 'Ba..."
2,E,[B-P-EC@BasicInfo-BasicDftGrn_idx],B-P-EC@BasicInfo-BasicDft,4,[expander-CateEmbed],"[{'vocab_tokenizer': {'_padding': 0, 'X': 1, '...","[{'nn_type_nn_name': 'expander-CateEmbed', 'Ba..."
3,E,[B-P-EC@DT_min-DTDftGrn_idx],B-P-EC@DT_min-DTDft,4,[expander-CateEmbed],"[{'vocab_tokenizer': {'_padding': 0, '_missing...","[{'nn_type_nn_name': 'expander-CateEmbed', 'Ba..."
4,E,[B-P-EC-A1C@DT-DTDftGrn_idx],B-P-EC-A1C@DT-DTDft,5,[expander-CateEmbed],"[{'vocab_tokenizer': {'_padding': 0, '_missing...","[{'nn_type_nn_name': 'expander-CateEmbed', 'Ba..."
5,E,[B-P-EC-A1C@V-A1CNumeDftGrn_wgt],B-P-EC-A1C@V-A1CNumeDft,5,[expander-NumeEmbed],"[{'vocab_tokenizer': [1, 1, 1, 1, 1, 1, 1, 1, ...","[{'nn_type_nn_name': 'expander-NumeEmbed', 'Ba..."
6,E,[B-P-EC-Diag@DT-DTDftGrn_idx],B-P-EC-Diag@DT-DTDft,5,[expander-CateEmbed],"[{'vocab_tokenizer': {'_padding': 0, '_missing...","[{'nn_type_nn_name': 'expander-CateEmbed', 'Ba..."
7,E,[B-P-EC-Diag@Value-DiagDftGrn_idx],B-P-EC-Diag@Value-DiagDft,5,[expander-CateEmbed],"[{'vocab_tokenizer': {'_padding': 0, '_missing...","[{'nn_type_nn_name': 'expander-CateEmbed', 'Ba..."
8,E,[B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzL...,B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLM,7,[expander-LLMEmbed],[{'vocab_tokenizer': PreTrainedTokenizerFast(n...,"[{'nn_type_nn_name': 'expander-LLMEmbed', 'Bas..."


In [43]:
from fieldnn.module.embedblock import EmbedBlockLayer

EmbedBlock = EmbedBlockLayer(df_SubUnit)

RECFLD_TO_EMBEDTESNOR = EmbedBlock(RECFLD_TO_TENSOR)

for k, v in RECFLD_TO_EMBEDTESNOR.items():
    print(k, v['info'].shape)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


B-P@age-AgeNumeDft torch.Size([4, 19, 128])
B-P@basicInfo-basicInfoDft torch.Size([4, 2, 128])
B-P-EC@BasicInfo-BasicDft torch.Size([4, 25, 2, 128])
B-P-EC@DT_min-DTDft torch.Size([4, 25, 7, 128])
B-P-EC-A1C@DT-DTDft torch.Size([4, 25, 1, 7, 128])
B-P-EC-A1C@V-A1CNumeDft torch.Size([4, 25, 1, 37, 128])
B-P-EC-Diag@DT-DTDft torch.Size([4, 25, 22, 7, 128])
B-P-EC-Diag@Value-DiagDft torch.Size([4, 25, 22, 3, 128])
B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLM torch.Size([4, 25, 1, 14, 121, 11, 128])


## ReprBlock

In [44]:
full_recfldgrn_list = [i for i in RECFLD_TO_EMBEDTESNOR]
full_recfldgrn_list

['B-P@age-AgeNumeDft',
 'B-P@basicInfo-basicInfoDft',
 'B-P-EC@BasicInfo-BasicDft',
 'B-P-EC@DT_min-DTDft',
 'B-P-EC-A1C@DT-DTDft',
 'B-P-EC-A1C@V-A1CNumeDft',
 'B-P-EC-Diag@DT-DTDft',
 'B-P-EC-Diag@Value-DiagDft',
 'B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLM']

In [45]:
from fieldnn.dataflowfn.reprflowfn import get_Repr_dataflow_table
from fieldnn.dataflowfn.reprflowfn import update_df_Repr_dataflow
from fieldnn.dataflowfn.reprflowfn import update_df_Repr_dataflow_completename
from fieldnn.dataflowfn.reprflowfn import get_Repr_SubUnit_List

df_dataflow = get_Repr_dataflow_table(full_recfldgrn_list)
# df_dataflow_new = update_df_Repr_dataflow(df_dataflow, style = 'ReducerOnly')
# df_dataflow_new

df_dataflow_new = update_df_Repr_dataflow(df_dataflow, style = 'Reducer&Merger')
df_dataflow = df_dataflow_new.copy()
df_SubUnit = get_Repr_SubUnit_List(df_dataflow)
df_SubUnit

,SubUnitName,input_names,output_name,input_layerid,output_layerid
0,R,[B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk],B-P-EC-PN-PNSect-PNSectSent,7,6
1,R,[B-P-EC-PN-PNSect-PNSectSent],B-P-EC-PN-PNSect,6,5
2,R,[B-P-EC-A1C@DT-DTDft],B-P-EC-A1C@DT,5,4
3,R,[B-P-EC-A1C@V-A1CNumeDft],B-P-EC-A1C@V,5,4
4,R,[B-P-EC-Diag@DT-DTDft],B-P-EC-Diag@DT,5,4
5,R,[B-P-EC-Diag@Value-DiagDft],B-P-EC-Diag@Value,5,4
6,R,[B-P-EC-PN-PNSect],B-P-EC-PN,5,4
7,MR,"[B-P-EC-A1C@DT, B-P-EC-A1C@V]",B-P-EC-A1C,4,4
8,MR,"[B-P-EC-Diag@DT, B-P-EC-Diag@Value]",B-P-EC-Diag,4,4
9,R,[B-P-EC@BasicInfo-BasicDft],B-P-EC@BasicInfo,4,3


In [46]:
from fieldnn.dataflowfn.baseflowfn import mapping_SubUnitName_to_SubUnitNNList
from fieldnn.dataflowfn.baseflowfn import generate_BasicNN_Config
from fieldnn.dataflowfn.baseflowfn import get_SubUnit_Default_NNPara_List
from fieldnn.dataflowfn.baseflowfn import get_SubUnit_BasicNN_Config_List


############################################# Hyperparameters
default_BasicNNtype_To_NNName = {
    'expander': None, # will be updated according to the Grn Type
    'reducer': 'Max',
    'merger': 'Merger',
    'learner': None, # TODO: ignore this currently
    
}
#############################################

s = df_SubUnit.apply(lambda x: mapping_SubUnitName_to_SubUnitNNList(x['SubUnitName'], 
                                                                    x['input_names'],
                                                                    default_BasicNNtype_To_NNName), 
                    axis = 1)


df_SubUnit['SubUnit_BasicNN_List'] = s
# df_SubUnit


fldgrn_folder = 'data/ProcData/FldGrnInfo'
learner_default_dict = {} # To update it in the future. 


s = df_SubUnit.apply(lambda x: get_SubUnit_Default_NNPara_List(x['SubUnit_BasicNN_List'], 
                                                               x['input_names'],
                                                               fldgrn_folder, 
                                                               learner_default_dict), axis = 1)

df_SubUnit['SubUnit_DefaultBasicNN_List'] = s

############################
embed_size = 128
process = {'activator': 'gelu',
           'dropout': {'p': 0.5, 'inplace': False},
           'layernorm': {'eps': 1e-05, 'elementwise_affine': True}}
############################

s = df_SubUnit.apply(lambda x: get_SubUnit_BasicNN_Config_List(x['SubUnit_BasicNN_List'], 
                                                               x['SubUnit_DefaultBasicNN_List'], 
                                                               x['input_names'], 
                                                               x['output_name'], 
                                                                embed_size, 
                                                                process, 
                                                               ), axis = 1)

s

df_SubUnit['SubUnit_BasicNN_Config_List'] = s
df_SubUnit

,SubUnitName,input_names,output_name,input_layerid,output_layerid,SubUnit_BasicNN_List,SubUnit_DefaultBasicNN_List,SubUnit_BasicNN_Config_List
0,R,[B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk],B-P-EC-PN-PNSect-PNSectSent,7,6,[reducer-Max],[{}],"[{'nn_type_nn_name': 'reducer-Max', 'Basic_Con..."
1,R,[B-P-EC-PN-PNSect-PNSectSent],B-P-EC-PN-PNSect,6,5,[reducer-Max],[{}],"[{'nn_type_nn_name': 'reducer-Max', 'Basic_Con..."
2,R,[B-P-EC-A1C@DT-DTDft],B-P-EC-A1C@DT,5,4,[reducer-Max],[{}],"[{'nn_type_nn_name': 'reducer-Max', 'Basic_Con..."
3,R,[B-P-EC-A1C@V-A1CNumeDft],B-P-EC-A1C@V,5,4,[reducer-Max],[{}],"[{'nn_type_nn_name': 'reducer-Max', 'Basic_Con..."
4,R,[B-P-EC-Diag@DT-DTDft],B-P-EC-Diag@DT,5,4,[reducer-Max],[{}],"[{'nn_type_nn_name': 'reducer-Max', 'Basic_Con..."
5,R,[B-P-EC-Diag@Value-DiagDft],B-P-EC-Diag@Value,5,4,[reducer-Max],[{}],"[{'nn_type_nn_name': 'reducer-Max', 'Basic_Con..."
6,R,[B-P-EC-PN-PNSect],B-P-EC-PN,5,4,[reducer-Max],[{}],"[{'nn_type_nn_name': 'reducer-Max', 'Basic_Con..."
7,MR,"[B-P-EC-A1C@DT, B-P-EC-A1C@V]",B-P-EC-A1C,4,4,"[merger-Merger, reducer-Max]","[{}, {}]","[{'nn_type_nn_name': 'merger-Merger', 'Basic_C..."
8,MR,"[B-P-EC-Diag@DT, B-P-EC-Diag@Value]",B-P-EC-Diag,4,4,"[merger-Merger, reducer-Max]","[{}, {}]","[{'nn_type_nn_name': 'merger-Merger', 'Basic_C..."
9,R,[B-P-EC@BasicInfo-BasicDft],B-P-EC@BasicInfo,4,3,[reducer-Max],[{}],"[{'nn_type_nn_name': 'reducer-Max', 'Basic_Con..."


In [47]:
# from fieldnn.module.reprblock import ReprBlockLayer
ReprBlock = ReprBlockLayer(df_SubUnit)

In [48]:
fld_updates_dict = {}
for i in RECFLD_TO_EMBEDTESNOR:
    layernum = len(i.split('-'))
    fld = i.split('-')[-1]
    if '@' in fld:
        # print(fld)
        neat_i = '-'.join(i.split('-')[:-1]) + '-' + fld.split('@')[0]
        # print(neat_i)
        same_neat_list = [t for t in RECFLD_TO_EMBEDTESNOR if neat_i + '@' in t]
        # print(same_neat_list)
        if len(same_neat_list) == 1: # itself
            # RECFLD_TO_EMBEDTESNOR[]
            fld_updates_dict[i] = neat_i
            
for old, new in fld_updates_dict.items():
    RECFLD_TO_EMBEDTESNOR[new] = RECFLD_TO_EMBEDTESNOR.pop(old)
    
    
[i for i in RECFLD_TO_EMBEDTESNOR]

['B-P@age-AgeNumeDft',
 'B-P@basicInfo-basicInfoDft',
 'B-P-EC@BasicInfo-BasicDft',
 'B-P-EC@DT_min-DTDft',
 'B-P-EC-A1C@DT-DTDft',
 'B-P-EC-A1C@V-A1CNumeDft',
 'B-P-EC-Diag@DT-DTDft',
 'B-P-EC-Diag@Value-DiagDft',
 'B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk']

In [49]:
[i for i in RECFLD_TO_EMBEDTESNOR]

['B-P@age-AgeNumeDft',
 'B-P@basicInfo-basicInfoDft',
 'B-P-EC@BasicInfo-BasicDft',
 'B-P-EC@DT_min-DTDft',
 'B-P-EC-A1C@DT-DTDft',
 'B-P-EC-A1C@V-A1CNumeDft',
 'B-P-EC-Diag@DT-DTDft',
 'B-P-EC-Diag@Value-DiagDft',
 'B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk']

In [50]:
[i for i in RECFLD_TO_EMBEDTESNOR]

['B-P@age-AgeNumeDft',
 'B-P@basicInfo-basicInfoDft',
 'B-P-EC@BasicInfo-BasicDft',
 'B-P-EC@DT_min-DTDft',
 'B-P-EC-A1C@DT-DTDft',
 'B-P-EC-A1C@V-A1CNumeDft',
 'B-P-EC-Diag@DT-DTDft',
 'B-P-EC-Diag@Value-DiagDft',
 'B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk']

In [51]:
OUTPUT_TO_TENSOR = ReprBlock(RECFLD_TO_EMBEDTESNOR)
[i for i in OUTPUT_TO_TENSOR]

['B-P@age-AgeNumeDft',
 'B-P@basicInfo-basicInfoDft',
 'B-P-EC@BasicInfo-BasicDft',
 'B-P-EC@DT_min-DTDft',
 'B-P-EC-A1C@DT-DTDft',
 'B-P-EC-A1C@V-A1CNumeDft',
 'B-P-EC-Diag@DT-DTDft',
 'B-P-EC-Diag@Value-DiagDft',
 'B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk',
 'B-P-EC-PN-PNSect-PNSectSent',
 'B-P-EC-PN-PNSect',
 'B-P-EC-A1C@DT',
 'B-P-EC-A1C@V',
 'B-P-EC-Diag@DT',
 'B-P-EC-Diag@Value',
 'B-P-EC-PN',
 'B-P-EC-A1C',
 'B-P-EC-Diag',
 'B-P-EC@BasicInfo',
 'B-P-EC@DT_min',
 'B-P-EC@A1C',
 'B-P-EC@Diag',
 'B-P-EC@PN',
 'B-P-EC',
 'B-P@age',
 'B-P@basicInfo',
 'B-P@EC',
 'B-P']